# Chapter 2: Discuss – a Hacker News Clone

In this chapter, we'll be creating a web app similar to Hacker News or Reddit, where
users can share and discuss links to web content. We'll call the application Discuss.
To keep things simple, we'll be emulating the minimalistic look of Hacker News,
which is text only and has a very simple interface. Reddit, on the other hand, is much
more visually rich and has a lot of extra features that we won't be adding to our site.

Here's an outline of what we'll be covering in this chapter:
* Allowing users to submit their own content
* Allowing users to vote on content submitted by other users
* Ranking the user-submitted content based on simple algorithms
* Preventing spammers from abusing our site using captchas

## Chapter code packs

If you have developed a few Django applications, you'll probably know that for most
applications, a lot of the code and configurations that you do when starting out is the
same. You set up the database in the same way, maybe changing the **Database (DB)**
name and user/pass pairs, you set up your media, static URL, and root paths, and
then you add user authentication using the built-in `auth contrib` application and
the provided views, only creating minimal templates that are good enough to get the
job done in the start.

Walking you through the basic setup at the start of every chapter would be very
boring—both for you to read and for me to write. Instead, I've provided what I call
**Code Packs**. These are `zip` files that contain the Django application already set up so
that we can jump straight to the interesting parts of the code instead of having to go
through the tedious setup process again and again.

Don't worry, I won't skip any new Django features that we haven't looked at yet. Each
code pack contains code that has already been explained to you in previous chapters.
For instance, the code pack for this chapter contains a Django application that has the
user registration, login, and logout views, templates, and URLs already set up. This is
the stuff that we have already looked at in detail in the previous chapter.

To use these code packs, you will need to download them, unzip them in the project
root folder, and create a virtual environment for them. Then, you'll need to run the
following command to have Django installed in your new virtual environment:

```sh
> pip install django
> python manage.py migrate
```

Once you've done all these steps, you'll be ready to start working on the fun parts
of the application. For all the following chapters, I've given you the link to the code
pack and I assume that you have already extracted and set up a virtual environment
for it.

> tips

> If you are not sure how to use the code packs, there is `Readme.txt`
in each ZIP file. You should read this to figure out how to get started
using the code pack.

## Requirements 

For any complex application, it is always a good idea to know what features we
need to work on before we jump into coding. Let's look at what we want to create
in this chapter.

We want to have a Django-based link sharing and discussion site like Hacker News.
The site should have user accounts, allow users to share links, have a page where
these links are listed in some defined order, and allow users to vote and comment on
those links.

Additionally, we'd like to have safeguards against spammers and malicious users
who would degrade the content quality of our site if left unchecked.

In a list form, here are the features that we want our application to provide:
* User registration and authentication (already provided in the code pack)
* Link submission by users
* Voting on links submitted by other users
* Commenting on the submissions and replying to comments by other users
* An algorithm to rank the submitted links in some defined order that
depends on a number of factors including the votes for that link, number of
comments, and age of the submission
* A way to disallow spammers from creating scripts that can automatically
flood our site with submissions

## Starting out

By now you should have the application ready to test if you've followed the
instructions given at the start of the chapter. Let's see what it looks like at the
moment. Start the application by running the following command in your terminal
from the application folder. You'll need to make sure that your virtual environment
is activated before you can run this command:

```sh
> python manage.py runserver
```

Open `http://127.0.0.1:8000` in your browser and you should see the following
basic page:

![Welcome to Discuss](img/Discuss_Welcome.png)

As you can see, we have links for **Login** and **Create New Account**. You should go
ahead and create a new account. Log in with this account and you'll see the two links
replaced with a **Logout** link. This is the basic application setup that we'll use moving
forward. You should make sure that you are able to use the application at this point,
as all further development will be built on top of this.

## Link submission

Let's look at what features we want to be related with link submissions. This is just a
part of the features list that we saw at the start of the chapter:
* Link submission by users
* Voting on links submitted by other users
* Commenting on the submissions and replying to comments by other users

Let's think about what models we'll need to implement this. First, we need a model
to hold information about a single submission, such as the title, URL, who submitted
the link, and at what time. Next, we need a way to track votes on submissions by
users. This can be implemented by a `ManyToMany` field from the submission model to
the `User` model. This way, whenever a user votes for a submission, we just add them
to the set of related objects and remove them if they decide to take back their vote.

Commenting as a feature is separate from link submissions because it can be
implemented as a separate model that links to the submission model with
ForeignKey. We'll look at commenting in the next section. For now, we'll
concentrate on link submissions.

To start out, let's create a new application in our project for link submission-related
features. Run the following command in your CLI:

```sh
> python manage.py startapp links
```

Then, add our newly created app to the INSTALLED_APPS settings variable. Now
we're ready to write code.

Let's start with the models. Here's the code for `Link` model. This code should be in
`links/models.py`:

```python
from django.contrib.auth.models import User
from django.db import models


class Link(models.Model):
    title = models.CharField(max_length=100)
    url = models.URLField()
    submitted_by = models.ForeignKey(User)
    upvotes = models.ManyToManyField(User, related_name='votes')
    submitted_on = models.DateTimeField(auto_now_add=True, editable=False)
```

Note that we had to set `related_name` for the `upvotes` field. If we hadn't done this,
we would get an error from Django when we try to run our application. Django
would have complained about having two relationships to the `User` model from the
Link model, both trying to create a reverse relationship named link. To fix this, we
explicitly named the reverse relationship from the `User` model to the `Link` model via
the `upvotes` field. The `User` model should now have an attribute called votes, which
can be used to get a list of submissions that the user has voted on.

Once you've saved this code, you'll need to make and run migrations in order for
Django to create database tables for the new model. To do so, type the following
commands:

```sh
> python manage.py makemigrations
> python manage.py migrate
```

Next, let's work on the templates and views. We'll customize the generic `CreateView`
that we've seen in the previous chapter for the view. Put this code in `links/views.py`:

```python
from django.contrib.auth.decorators import login_required
from django.core.urlresolvers import reverse
from django.http.response import HttpResponseRedirect
from django.utils.decorators import method_decorator
from django.views.generic import CreateView
from links.models import Link


class NewSubmissionView(CreateView):
    model = Link
    fields = ('title', 
           'url')
    template_name = 'new_submission.html'

    @method_decorator(login_required)
    def dispatch(self, *args, **kwargs):
        return super(NewSubmissionView, self).dispatch(*args, **kwargs)

    def form_valid(self, form):
        new_link = form.save(commit=False)
        new_link.submitted_by = self.request.user
        new_link.save()

        self.object = new_link
        return HttpResponseRedirect(self.get_success_url())

    def get_success_url(self):
        return reverse('home')
```

This should look familiar to the `CreateView` subclasses that we have already created
in the previous chapter. However, look closer! This time, we don't define a custom
form class. Instead, we just point to the model—`Link` in this case—and `CreateView`
automagically creates a model form for us. This is the power of built-in Django
generic views. They give you multiple options to get what you want, depending on
how much customization you need to do.

We define the `model` and `fields` attributes. The `model` attribute is self-explanatory.
The `fields` attribute has the same meaning here as it has in a `ModelForm` subclass.
It tells Django which fields we want to be made editable. In our `link` model, the title
and submission URL are the only two fields that we want the user to control, so we
put these in the fields list.

Another important thing to look at here is the `form_valid` function. Note that it
doesn't have any calls to super. Unlike our previous code, where we always called
the parent class method for methods that we had overridden, we do no such thing
here. That's because `form_valid` of` CreateView` calls the `save()` method of the
form. This will try to save the new link object without giving us the chance to set its
submitted_by field. As the submitted_by field is required and can't be null, the
object won't be saved and we'll have to deal with a database exception.

So instead, we chose to not call the `form_valid` method on the parent class and
wrote the code for it ourselves. To do so, I needed to know what the base method
did. So I looked up the documentation for it at <https://docs.djangoproject.com/en/1.9/ref/class-based-views/mixins-editing/#django.views.generic.edit.ModelFormMixin.form_valid>:

> *"Saves the form instance, sets the current object for the view, and redirects to
get_success_url()."*

If you look at our code for the `form_valid` function, you will see that we do
exactly the same thing. If you're ever faced with a similar situation, the Django
documentation is the best resource to clear things up. It has some of the best
documentation that I have ever encountered in any of the open source projects
that I have used.

Finally, we need the template and URL configuration for the link submission feature.
Create a new folder called `templates` in the `links` directory and save this code in a
file called `new_submission.html`:

```html
{% extends "base.html" %}

{% block content %}
    <h1>New Submission</h1>
    <form action="" method="post">{% csrf_token %}
        {{ form.as_p }}
        <input type="submit" value="Submit" />
    </form>
{% endblock %}
```

In `discuss/urls.py`, import the new view:

```python
from links.views import NewSubmissionView
```

Create a new URL configuration for this view:

```python
url(r'^new-submission/$', NewSubmissionView.as_view(), name='newsubmission'),
```

That's it. All the code that we need to get a basic link submission process up is
written. However, to be able to test it out, we'll need to give the user some way of
accessing this new view. The navigation bar in our base.html template seems like a
good place to put the link in for this. Change the code for the `nav` HTML tag in `base.html` in the `templates` directory in the project root to match the following code:

```html
<nav>
    <ul>
        {% if request.user.is_authenticated %}
        <li><a href="{% url "new-submission" %}">Submit New Link</a></li>
        <li><a href="{% url "logout" %}">Logout</a></li>
        {% else %}
        <li><a href="{% url "login" %}">Login</a></li>
        <li><a href="{% url "user-registration"%}">Create New Account</a></li>
        {% endif %}
    </ul>
</nav>
```

To test it out, run the development server and open the home page. You'll see a
**Submit New Link** option in the navigation menu on the top. Click on it and you'll
see a page similar to the following one. Fill in the data and click on submit. If the
data that you've filled in doesn't have any errors, you should be redirected to the
home page.

![New Submission](img/Discuss_NewSubmission.png)

While this works, this isn't the best user experience. Redirecting the user to the home
page without giving them any feedback on whether their link was submitted is not
good. Let's fix this next. We'll create a details page for the submissions and if the user
was successful in submitting a new link, we'll take them to the details page.

Let's start with the view. We'll use the DetailView generic view provided by Django.
In your `links/views.py` file, import DetailView:

```python
from django.views.generic import DetailView
```

Subclass it for our submission detail view:

```python
class SubmissionDetailView(DetailView):
    model = Link
    template_name = 'submission_detail.html'
```

Create the `submission_detail.html` template in the `links/templates` directory
and put in the following Django template code:

```html
{% extends "base.html" %}

{% block content %}
<h1><a href="{{ object.url }}" target="_blank">{{ object.title}}</a></h1>
<p>submitted by: <b>{{ object.submitted_by.username }}</b></p>
<p>submitted on: <b>{{ object.submitted_on }}</b></p>
{% endblock %}
```

Configure the URL for this view in `discuss/urls.py` by first importing it:

```python
from links.views import SubmissionDetailView
```

Then, add a URL pattern for it to the `urlpatterns` list:

```python
url(r'^submission/(?P<pk>\d+)/$', SubmissionDetailView.as_view(),
name='submission-detail'),
```

Finally, we'll need to edit the `NewSubmissionView` `get_success_url` method to
redirect the user to our new detail view on successfully creating a new submission:

```python
def get_success_url(self):
    return reverse('submission-detail', kwargs={'pk': self.object.pk})
```

That's it. Now when you create a new submission, you should see the following
detail page for your new submission:

![Discuss Test 2](img/Discuss_Test2.png)

Now that link submission is done, let's look at implementing the comments feature.

## Comments 

We want our logged in users to be able to comment on submissions. We'd also like
users to reply to comments by other users. To achieve this, our comment model needs
to be able to track the submission it was made on and also have a link to its parent
comment (if it was made in reply to some other user's comment).

If you have ever used forums on the Internet, the way our comments section works
should seem familiar. One complaint that I've always had with all these forums is
that they allow this hierarchy of comments to go on forever. Then you end up with
10-level deep comments that extend off the screen:

```
Comment 1
    Comment 2
        Comment 3
            Comment 4
                Comment 5
                    Comment 6
                        Comment 7
                            Comment 8
                                Comment 9
                                    Comment 10
```

While there are a number of ways to solve this, the simplest probably is to cut off
nested replies beyond a certain level. In our case, no comments can be replies to
`Comment 2`. Instead, they must all be in reply to `Comment 1` or the parent submission.
This will make the implementation easier as we'll see later.

From our discussion so far, we know that our comment model will need foreign keys
to our submission models and also to itself in order to refer to parent comments.
This self reference, or a recursive relationship as Django documentation calls it, is
something that I have used maybe once in the five years (and more) I have been
creating web apps in Django. It's not something that is needed very often but
sometimes it results in elegant solutions, like the one you'll see here.

To keep things simple, we'll first implement commenting on link submissions and
later add code to handle replying to comments. Let's start with the model. Add the
following to `links/models.py`:

```python
class Comment(models.Model):
    body = models.TextField()
    commented_on = models.ForeignKey(Link)
    in_reply_to = models.ForeignKey('self', null=True)
    commented_by = models.ForeignKey(User)
    created_on = models.DateTimeField(auto_now_add=True, editable=False)
```    

The `in_reply_to` field here is the recursive foreign key that allows us to create a
hierarchy of comments and replies to them. As you can see, creating a recursive
foreign key is achieved by giving the model name `self` instead of the model name
like you would usually do with a normal foreign key.

Create and run the migrations to add this model to our database:

```sh
> python manage.py makemigrations
> python manage.py migrate
```

Next, let's think about the view and template. As we're only implementing
commenting on submissions for now, it makes sense that the form to create a new
comment also be visible on the submission details page. Let's create the form first.
Create a new `links/forms.py` file and add the following code:

```python
from django import forms
from links.models import Comment


class CommentModelForm(forms.ModelForm):
    link_pk = forms.IntegerField(widget=forms.HiddenInput)

    class Meta:
        model = Comment
        fields = ('body',)
```

We will just create a simple model form for the `Comment` model and add one extra
field that we'll use to keep track of which link the comment needs to be associated
with. To make the form available to our submission details template, import the form
in `links/views.py` by adding the following to the top of the file:

```python
from links.forms import CommentModelForm
```

We'll also add code to show comments for a submission on the details page now. So
we need to import the `Comment` model in the views file. Right after the line importing
the form, add another line of code importing the model:

```python
from links.models import Comment
```

To be able to display the comments associated with a submission and the form
to create a new submission, we'll need to make these two things available in the
template context of the submission details page. To do so, add a `get_context_data`
method to `SubmissionDetailView`:

```python
def get_context_data(self, **kwargs):
    ctx = super(SubmissionDetailView, self).get_context_data(**kwargs)
    
    submission_comments = Comment.objects.filter(commented_on=self.object)
    ctx['comments'] = submission_comments
    ctx['comment_form'] = CommentModelForm(initial={'link_pk': self.object.pk})

return ctx
```


We'll look at the initial attribute that we are passing to `CommentModelForm` in a
while. We'll also need to create a view where the new comment form is submitted.
Here's the code that you'll need to add it to `links/views.py`:

```python
class NewCommentView(CreateView):
    form_class = CommentModelForm
    http_method_names = ('post',)
    template_name = 'comment.html'

    @method_decorator(login_required)
    def dispatch(self, *args, **kwargs):
        return super(NewCommentView, self).dispatch(*args, **kwargs)

    def form_valid(self, form):
        parent_link = Link.objects.get(pk=form.cleaned_data['link_pk'])
        new_comment = form.save(commit=False)
        new_comment.commented_on = parent_link
        new_comment.commented_by = self.request.user
        new_comment.save()

        return HttpResponseRedirect(reverse('submission-detail', kwargs={'pk': parent_link.pk}))

    def get_initial(self):
        initial_data = super(NewCommentView, self).get_initial()
        initial_data['link_pk'] = self.request.GET['link_pk']

    def get_context_data(self, **kwargs):
        ctx = super(NewCommentView, self).get_context_data(**kwargs)
        ctx['submission'] = Link.objects.get(pk=self.request.GET['link_pk'])
        return ctx
```    

Even though we show the form on the submission detail page, in case the user inputs
incorrect data when submitting the form, such as pressing the submit button with an
empty body, we need a template that can show the form again along with the errors.
Create the `comment.html` template in `links/templates`:

```html
{% extends "base.html" %}

{% block content %}
    <h1>New Comment</h1>
    <p>
        <b>You are commenting on</b>
        <a href{% url 'submission-detail' pk=submission.pk %}">{{ submission.title }}</a>
    </p>
    <form action="" method="post">{% csrf_token %}
        {{ form.as_p }}
    <input type="submit" value="Post Comment" />
    </form>
{% endblock %}
```

You should already know what most of the code for the `CreateView` subclass does.
One thing that is new is the `get_inital` method. We'll look at it in detail later. For
now, let's get the comments feature up and running.

Let's add our new view to `discuss/urls.py`. First, import the view:

```python
from links.views import NewCommentView
```

Then, add it to the URL patterns:

```python
url(r'new-comment/$', NewCommentView.as_view(), name='new-comment'),
```

Finally, change `links/templates/submission_detail.html` to the following:

```html
{% extends "base.html" %}

{% block content %}
    <h1><a href="{{ object.url }}" target="_blank">{{ object.title
    }}</a></h1>
    <p>submitted by: <b>{{ object.submitted_by.username }}</b></p>
    <p>submitted on: <b>{{ object.submitted_on }}</b></p>

    <p>
        <b>New Comment</b>
        <form action="{% url "new-comment" %}?link_pk={{ object.pk }}"
        method="post">{% csrf_token %}
            {{ comment_form.as_p }}
            <input type="submit" value="Comment" />
        </form>
    </p>
    <p>
        <b>Comments</b>
        <ul>
            {% for comment in comments %}
            <li>{{ comment.body }}</li>
            {% endfor %}
        </ul>
    </p>
{% endblock %}
```

If you noticed the form action URL in our template, you'll see that we have added
the `link_pk` GET parameter to it. If you refer back to the code that you wrote for
NewCommentView, you'll see that we use this parameter value in the `get_context_data` and `get_inital` functions to get the `Link` object that the user is commenting on.

> tips

> I'm saving describing what the get_initial method
does until the next section when we get to adding replies to
comments.

Let's look at what we've made till now. Start the application using the `runserver`
command, open the home page in your browser, and then log in. As we don't yet
have any way to access old submissions, we'll need to create a new submission.
Do that and you'll see the new detail page. It should look similar to the following
screenshot:

![Discuss New Comment](img/Discuss_NewComment.png)

Add a comment and it should appear on the same page. Here's a screenshot with a
few comments added:

![Discuss New Comment 2](img/Discuss_NewComment2.png)

If you leave the body empty and press the **Comment** button, you should see the
comment template that you created earlier with an error message:

![Discuss New Comment 3](img/Discuss_NewComment3.png)

With basic commenting on submission working, let's look at how we'll implement
replying to comments. As we've already seen, our comment model has a field to
indicate that it was made in reply to another comment. So all we have to do in order
to store a comment as a reply to another comment is correctly set the `in_reply_to`
field. Let's first modify our model form for the Comment model to accept in addition
to a `link_pk`, a `parent_comment_pk` as well to indicate which (if any) comment is
the new comment a reply to. Add this field to `CommentModelForm` right after the
`link_pk` field:

```python
parent_comment_pk = forms.IntegerField(widget=forms.HiddenInput,
required=False)
```

Now we need a place to show a form to the user to post his reply. We could show
one form per comment on the submission details page, but that would end up
making the page look very cluttered for any submission with more than a few
comments. In a real-world project, we'd probably use JavaScript to generate a form
dynamically when the user clicks on the reply link next to a comment and submits
this. However, right now we are more focused on the Django backend and thus we'll
come up with another way that doesn't involve a lot of frontend work.

A third way, which we'll be using here, is to have a little link next to each comment
that takes the user to a separate page where they can record their reply. Here's the
view for that page. Put this in `links/views.py`:

```python
class NewCommentReplyView(CreateView):
    form_class = CommentModelForm
    template_name = 'comment_reply.html'

    @method_decorator(login_required)
    def dispatch(self, *args, **kwargs):
        return super(NewCommentReplyView, self).dispatch(*args, **kwargs)

    def get_context_data(self, **kwargs):
        ctx = super(NewCommentReplyView, self).get_context_data(**kwargs)
        ctx['parent_comment'] = Comment.objects.get(pk=self.request.GET['parent_comment_pk'])
   
       return ctx

    def get_initial(self):
    initial_data = super(NewCommentReplyView, self).get_initial()
    link_pk = self.request.GET['link_pk']
    initial_data['link_pk'] = link_pk
    parent_comment_pk = self.request.GET['parent_comment_pk']
    initial_data['parent_comment_pk'] = parent_comment_pk
    
    return initial_data

    def form_valid(self, form):
        parent_link = Link.objects.get(pk=form.cleaned_data['link_pk'])
        parent_comment = Comment.objects.get(pk=form.cleaned_data['parent_comment_pk'])
        new_comment = form.save(commit=False)
        new_comment.commented_on = parent_link
        new_comment.in_reply_to = parent_comment
        new_comment.commented_by = self.request.user
        new_comment.save()

        return HttpResponseRedirect(reverse('submission-detail', kwargs={'pk': parent_link.pk}))
```        

By now, having used it multiple times, you should be comfortable with CreateView.
The only new part here is the get_initial method, which we also used in
`NewCommentView` previously. In Django, each form can have some initial data. This
is data that is shown when the form is **Unbound**. The boundness of a form is an
important concept. It took me a while to wrap my head around it, but it is quite
simple. In Django, a form has essentially two functions. It can be displayed in the
HTML code for a web page or it can validate some data.

A form is bound if you passed in some data for it to validate when you initialized an
instance of the form class. Let's say you have a `form` class called `SomeForm` with two
fields, name and city. Say you initialize an object of the form without any data:

```python
form = SomeForm()
```

You have created an unbound instance of the form. The form doesn't have any data
associated with it and so can't validate anything. However, it can still be displayed
on a web page by calling `{{ form.as_p }}` in the template (provided it was passed
to the template via the context). It will render as a form with two empty fields: name
and city.

Now let's say you pass in some data when initializing the form:

```python
form = SomeForm({'name': 'Jibran', 'city': 'Dubai'})
```

This creates a bound instance of the form. You can call `is_valid()` on this form
object and it will validate the passed data. You can also render the form in an HTML
template, just like before. However, this time, it will render the form with both the
fields having the values that you passed here. If, for some reason, the values that you
passed didn't validate (for example, if you left the value for the city field empty), the
form will display the appropriate error message next to the field with the invalid data.

This is the concept of bound and unbound forms. Now let's look at what the initial
data in a form is for. You can pass initial data to a form when initializing an instance
by passing it in the initial keyword parameter:

```python
form = SomeForm(initial={'name': 'Jibran'})
```

The form is still unbound as you did not pass in the data attribute (which is the first
non-keyword argument to the constructor) but if you render it now, the name field
will have the value `'Jibran'` while the city field will still be empty.

The confusion that I faced when I first learned of the initial data was why it was
required. I could just pass the same data dictionary as the data parameter and the
form would still only receive a value for one field. The problem with this is that
when you initialize a form with some data, it will automatically try to validate that
data. Assuming that the city field is a required field, if you then try to render the
form in a web page, it will display an error next to the city field saying that this is a
required field. The initial data parameter allows you to supply values for form fields
without triggering validation on that data.

In our case, `CreateView` calls the `get_initial` method to get the dictionary to use
it as the initial data for the form. We use the submission ID and parent comment ID
that we will pass in the URL parameters to create the initial values for the `link_pk`
and `parent_comment_pk` form fields. This way, when our form is rendered on the
HTML web page, it will already have values for these two fields. Looking at the
`form_valid` method, we then extract these two values from the form's `cleaned_data` attribute and use it to get the submission and parent comment to associate the
reply with.

The `get_context_data` method just adds the parent comment object to the context.
We use it in the template to tell the user which comment they are replying to. Let's
look at the template, which you need to create in `links/templates/comment_reply.html`:

```html
{% extends "base.html" %}

{% block content %}
    <h1>Reply to comment</h1>
    <p>
        <b>You are replying to:</b>
        <i>{{ parent_comment.body }}</i>
    </p>
    <form action="" method="post">{% csrf_token %}
        {{ form.as_p }}
        <input type="submit" value="Submit Reply" />
    </form>
{% endblock %}
```

Nothing fancy here. Note how we use the `parent_comment` object that we passed
in the `get_context_data` method of the view. It's good UI practice to make sure
that the user is always given relevant information about the action that they are
about to take.

Import our new view in `discuss/urls.py`:

```python
from links.views import NewCommentReplyView
```

Add this pattern to the URL patterns list:

```python
url(r'new-comment-reply/$', NewCommentReplyView.as_view(), name='newcomment-
reply'),
```

Lastly, we need to give the user a link to get to this page. As we discussed before,
we'll be putting a link called **Reply** next to each comment on the submission detail
page. To do so, note the following line in `links/templates/submission_detail.html`:

```html
<li>{{ comment.body }}</li>
```

Change it to the following:

```html
<li>{{ comment.body }} (<a href="{% url "new-comment-reply" %}?link_pk={{ object.pk }}&parent_comment_pk={{ comment.pk }}">Reply</a>)</li>
```

Note how we pass in the submission ID and parent comment ID using GET params
when creating the URL. We did this with the comment form on the submission page
as well. This is a common technique you'll use a lot when creating Django apps.
These are the same URL parameters that we used in the comment reply view to
populate the initial data for the form and access the parent comment object.

Let's try it out. Click on **Reply** on one of the comments on the submission detail
page. If you've closed the old submission details page, you can create a new
submission and add some comments to it. By clicking on the `Reply` link, you'll see a
new page with a form for the comment body. Enter some text here and click on the
`Submit` button. Remember the text that you entered. We'll be looking for this next in
the upcoming few steps. In my testing, I entered **Reply to Comment 1**. Let's see what
our submission details page looks with our new reply comment:

![Discuss: reply to comment](img/Discuss_Reply.png)

It seems like it worked. However, if you look closely, you'll notice that the reply
we made (in my case, the **Reply to Comment 1** text) is shown at the end of the
comments list. It should be shown after **Comment 1** and ideally indented a bit to the
right as well to indicate the hierarchy. Let's fix this. First, in the `get_context_data`
method of `SubmissionDetailView` in the `links/views.py` file, note this line:

```python
submission_comments = Comment.objects.filter(commented_on=self.object)
```

Change it to the following:

```python
submission_comments = Comment.objects.filter(commented_on=self.object,
in_reply_to__isnull=True)
```

What we've done here is include only the comments that don't have a parent
comment. We do this by getting only comments that have the `in_reply_to` field set
to NULL. If you save this change and refresh the submission detail page, you'll notice
that your reply comment will be gone. Let's bring it back. Modify `link/templates/submission_detail.html` and change the paragraph that shows the comments (the
one with the `for` loop over the comments list) to match the following:

```python
<p>
    <b>Comments</b>
    <ul>
        {% for comment in comments %}
        <li>
            {{ comment.body }} (<a href="{% url "new-comment-reply"
            %}?link_pk={{ object.pk }}&parent_comment_pk={{ comment.pk }}">Reply</
            a>)
            {% if comment.comment_set.exists %}
            <ul>
                {% for reply in comment.comment_set.all %}
                <li>{{ reply.body }}</li>
                {% endfor %}
            </ul>
            {% endif %}
        </li>
        {% endfor %}
    </ul>
</p>
```

The new part here is between the `if` tag. First, we use the reverse relationship that
is created by the foreign key to itself to see if this comment has any other comments
pointing to it. We know that the only comments pointing to this comment would be
replies to this comment. If there are, we then create a new list and print the body for
each of the replies. As we've already decided that we only allow replies to the first
level of comments, we don't create any links to let users reply to the replies. Once
you've save these changes, let's see what our submission details page looks now:

![Discuss New Comment](img/Discuss_NewComment4.png)

That's more like it! We now have a complete link submission and commenting
system. Awesome! Let's move on to the other features now.

## Voting 

We need to allow users to vote on submissions. To keep things simple, we'll only
allow upvotes. A user can indicate that they like a submission. There is no way to
indicate disapproval. This keeps the code and UI simple. We also want to ensure that
one user can upvote a submission only once and they can remove their upvotes if
they change their minds or upvoted a submission by mistake.

If you take another look at the `Link` model, you'll see we already have an upvotes
field, which is a **Machine to machine (M2M)** with the User model. This is the only
database entry we'll need to allow and keep track of `upvotes` by users. In order to
upvote a submission, users will click on a link next to the submission. Until now, we
were able to make do without a page to list all submissions. It's a good idea to create
one now so that we can access and upvote the various submissions. We can't keep
creating new submissions every time we want to test something.

First, create this view in `links/views.py`. Remember to import `TemplateView` from
`django.views.generic` first:

```python
class HomeView(TemplateView):
    template_name = 'home.html'

    def get_context_data(self, **kwargs):
        ctx = super(HomeView, self).get_context_data(**kwargs)
        ctx['submissions'] = Link.objects.all()
        
        return ctx
```

Next, change the template at `template/home.html` to the following:

```html
{% extends "base.html" %}

{% block content %}
<h1>Welcome to Discuss</h1>
<h2>Submissions</h2>
<ul>
    {% for submission in submissions %}
    <li>
        <a href="{{ submission.url }}" target="_blank">{{
        submission.title }}</a>
        <i><a href="{% url "submission-detail" pk=submission.pk
        %}">Comments</a></i>
    </li>
    {% endfor %}
</ul>
{% endblock %}
```

Import our new `HomeView` at the top of `discuss/urls.py` and note the home URL
configuration in `discuss/urls.py`:

```python
url(r'^$', TemplateView.as_view(template_name='home.html'),
name='home'),
```

Change the preceding code to this:

```python
url(r'^$', HomeView.as_view(), name='home'),
```

Finally, let's provide our users with a handy link to the home page in the navigation
bar. In the `base.html` template (in the `templates` directory in the project root), add
this as the first list element of the navigation list outside of the user authentication if
condition:

```html
<li><a href="{% url "home" %}">Home</a></li>
```

That's it. There's nothing new in this code. It's pretty easy to understand and
you should have a clear idea of what's happening here by now. Let's look at
the end result. If you now open the home page of our app by browsing to
`http://127.0.0.1:8000` in your browser, you should see something similar to the
following screenshot. Of course, your page will not be the same as this as you will
have added your own test content:

![Discussion Home](img/Discussion_Home.png)

You'll see a list of submissions. If you click on any, you'll have a new tab open
with the link for that submission. You'll also see a `Comments` link next to each
submission. Clicking on this takes you to the submission detail page.

Let's talk a bit about how we're going to implement the upvoting feature. The M2M
`upvotes` field that we created in the `Link` model should give you a hint. Whenever
a user `upvotes` a submission, we add them to this relationship. As an M2M
relationship ensures that if we add the same object multiple times it doesn't create a
new record, we easily ensure that one user can vote on a submission only once.

Let's create a view that adds the logged in user to the upvoters list of a submission
and then takes them back to the home page. We'll also add a link to each submission
on the home page that lets the user upvote the submission with this new view.

In `links/views.py`, import the `View` generic view class from `django.views.generic`, and then create this view:

```python
class UpvoteSubmissionView(View):
    def get(self, request, link_pk, **kwargs):
        link = Link.objects.get(pk=link_pk)
        link.upvotes.add(request.user)


        return HttpResponseRedirect(reverse('home'))
```

Next, import this new view in `discuss/urls.py` and add it to the URL patterns :

```python
url(r'^upvote/(?P<link_pk>\d+)/$', UpvoteSubmissionView.as_view(),
name='upvote-submission'),
```

In `templates/home.html`, add the **Upvote** link above the submission title link:

```html
<a href="{% url "upvote-submission" link_pk=submission.pk %}">Upvote</
a>
```

Open up the home page and you'll see an **Upvote** link next to each submission title.
Clicking on the link should bring you back to the home page. It should look similar
to the following screenshot:

If you upvote a link, it immediately brings you back to the home page without
any indication that your upvote was recorded. The fix for this is simple. Change
the Upvote link HTML line that you just added to the home page template to the
following:

```html
{% if request.user in submission.upvotes.all %}
    Upvoted
{% else %}
    <a href="{% url "upvote-submission" link_pk=submission.pk
    %}">Upvote</a>
{% endif %}
```

If you open up the home page again, you'll see a simple **Upvoted** text next to
submissions that you've already upvoted instead of the link that you saw before. We
should also allow the user to remove his upvote from a submission. First, create a
new view for this in `links/views.py`:

```python
class RemoveUpvoteFromSubmissionView(View):
    def get(self, request, link_pk, **kwargs):
        link = Link.objects.get(pk=link_pk)
        link.upvotes.remove(request.user)


        return HttpResponseRedirect(reverse('home'))
```

This is almost the same as the view we created to record a new upvote. The only
difference is that here we use the remove method of the related manager. Next, we'll
need to add this to the URLs file at `discuss/urls.py`. Import our new view here
and add the following URL configuration:

```python
url(r'^upvote/(?P<link_pk>\d+)/remove/$',
RemoveUpvoteFromSubmissionView.as_view(), name='remove-upvote'),
```

Finally, let's change the **Upvoted** label that we added to the home page before to be a
link to remove the upvote. In your `templates/home.html` file, note these lines:

```html
{% if request.user in submission.upvotes.all %}
    Upvoted
{% else %}
```

Change them to the following:

```html
{% if request.user in submission.upvotes.all %}
<a href="{% url "remove-upvote" link_pk=submission.pk %}">Remove Upvote</a>
{% else %}
```            

That's it! Now when you visit the home page, you'll see the **Remove Upvote** link for
all the submissions that you have already upvoted. By clicking on the link, you'll be
redirected back to the home page with your upvote removed. You should see the
**Upvote** link for that submission again as you can upvote it again.

## Ranking submission

The next feature on our list is ranking the submissions using an intelligent algorithm.
Let's take a look at what our feature description requires:

> tips

> An algorithm to rank the submitted links in some defined order that
depends on a number of factors including the votes for that link, the
number of comments and the age of the submission

We have all this information in our database. We need to create an algorithm that
will use all these pieces of information to give each submission a rank. Then, we'll
just sort the submission using this ranking and show them in the sorted order. To
keep things simple, let's use the following algorithm:

```
rank = number of votes + number of comments – number of days since
submission
```

Seems simple enough, except for maybe the number of days since submission
calculations. However, the `datetime` module in the Python standard library
makes this a breeze for us. In Python, if you subtract two `datetime` objects, you
get a `timedelta` object. This object represents the time difference between the two
`datetime` objects. It has an attribute called days that—as is obvious by the name—
holds the number of days between the two dates. We'll subtract the date that we get
from `datetime.datetime.now()` from the submitted_on field of the submission
and use the days attribute of the resulting `timedelta` object.

Let's plug this algorithm into our home view so that our submissions are listed by
their ranks. Change `HomeView` in `links/views.py` to match the following code:

```python
class HomeView(TemplateView):
    template_name = 'home.html'

    def get_context_data(self, **kwargs):
        ctx = super(HomeView, self).get_context_data(**kwargs)

        now = timezone.now()
        submissions = Link.objects.all()
    
        for submission in submissions:
            num_votes = submission.upvotes.count()
            num_comments = submission.comment_set.count()
            date_diff = now - submission.submitted_on
            number_of_days_since_submission = date_diff.days
        submission.rank = num_votes + num_comments - number_of_days_since_submission

        sorted_submissions = sorted(submissions, key=lambda x: x.rank, reverse=True)
        ctx['submissions'] = sorted_submissions

        return ctx
```

You will also need to import the `timezone` module from Django utilities using
the following:

```python
from django.utils import timezone
```

This is needed because Django uses something called `timezone` aware datetimes.
For more details on this, read the Django documentation for `timezone`
awareness at <https://docs.djangoproject.com/en/stable/topics/i18n/timezones/#naive-and-aware-datetime-objects>.

This new code might look a bit complicated but trust me, it's pretty simple. Let's look
at it one line at a time. The first thing we do is get the current date time using the
`timezone.now()` function. Next, we get all the submissions that we want to show on
the home page and start looping over them.

In the loop, we first calculate the number of votes and comments on the submission
using the `count()` method that you have seen before on Django querysets.
The only different thing here is that we use it on the queryset returned by the
`RelatedManager` object for the many-to-many upvotes field and the reverse
relationship to the comments model.

As explained before, we then calculate the number of days since the submission was
submitted to our site using the Python date arithmetic. Finally, we calculate and
assign the rank for the submission to the object.

After the loop ends, each of our `Link` objects has a rank attribute that holds the final
ranking for it. We then use the Python built-in `sorted` function to sort this list. When
you're working with lists, which is quite often in Python and Django, the sorted
function is something you will end up using many times. You should familiarize
yourself with its syntax and features by reading the documentation at <https://docs.python.org/3/library/functions.html#sorted>. Trust me, it's well worth
the time to read this documentation slowly and understand it completely. I have
used the sorted built-in function more times than I can remember. It's indispensable.

Finally, we assign the sorted submissions list to the submissions context variable.
As we are already using this variable in the home page template, we do not need
to change anything else beyond HomeView. If you open the home page now, you'll
see that the sorting order of the submissions has changed and now reflects our
new algorithm.

This is a good place to reflect upon the benefits of the modularity provided by the
Model-View-Template architecture used by Django. As you can see, we added quite
a significant feature, but we never had to change the URLs or templates for the
home page. As these are individual modules, we only changed the view code and
everything else still worked with our new and improved sorting order.

## Spam protection

The last feature that we want to have in our application is spam protection. We
want users to be able to post content on our site, but we want to prevent abuse by
spammers. Spamming, as you probably know, refers to malicious Internet users
posting inappropriate or irrelevant content to a site. Often, spammers use scripts
created specifically to target sites that allow user-submitted content, such as our web
app. While we can't stop spammers easily from submitting spam content to our site
manually, we can make sure that they are not able to use scripts to generate a lot of
spam with just a click of the mouse. Usually, if spammers can't use their scripts on
websites, they move on to easier targets.

The important concept that I want you to learn from this feature isn't how to
implement spam protection. That's something you need to decide based on the
requirements of your own project. What I'll be showing here is how to use open
source Django applications created by other developers to add features to your own
Django projects. This is an important concept that you should be familiar with. Most
of the time, if you're looking for ways to solve an issue while developing your web
app, a search of the Internet turns up a number of open source applications that
were developed by other programmers to solve the same issue. You get applications
solving all sizes of issues—from the smallest providing features such as a new type
of form field (for example, a calendar form field that uses a JavaScript calendar) to
large applications providing complete Django-based forums that you can integrate
with your Django website easily and provide users with an easy-to-use and goodlooking
forum.

We'll be using the ReCaptcha service from Google to provide us with a mechanism
to stop spammers. You can learn more about the service at <https://www.google.com/recaptcha>. You will also need to register for an account here and create an API
key. It will ask for a label, which I set to **Discuss Django Blueprints**, and a domain,
which I set to `127.0.0.1`. The `owners` field should have your e-mail address there.
Once you submit this form, you'll be presented with a screen that shows you your
public and private keys. Keep this page open as we'll use these values in a bit.

Next, we need to find a Django application that allows us to use the ReCaptcha
service. A Google search led me to <https://github.com/praekelt/djangorecaptcha>.
It seems like a well-maintained and simple solution to our problem.
In order to use it, we first have to install it in our virtual environment. On your
command line, make sure that you have the virtual environment active. Then,
install this package using the following `pip` command:

```sh
> pip install django-recaptcha
```

This will install the package. Next, add captcha to the list of `INSTALLED_APPS`
in your `discuss/settings.py` file. Also, add the `RECAPTCHA_PUBLIC_KEY` and
`RECAPTCHA_PRIVATE_KEY` variables to the settings file. Set their values to the
appropriate keys that you were given on the Google ReCaptcha API keys page that
I asked you to keep open before. **Site Key** is the public key and **Secret Key**
is the
private key. Finally, in your `settings.py` file, set the following variable:

```python
NOCAPTCHA = True
```

The setup is ready. We're ready to use ReCaptcha in our forms. For demonstration,
I'll only add it to the comment form that you can see on the submission detail page.
Open up `links/forms.py` and add this import to the top:

```python
from captcha.fields import ReCaptchaField
```

Then, add this field to `CommentModelForm`:

```python
captcha = ReCaptchaField()
```

That's it! You have successfully added Google `ReCaptcha` to your website! Let's try
it out. Open up the details page for any submission and now, right below the body
field that we had previously for the comment, you'll see the Google `ReCaptcha` box
as well:

![Discussion ReCaptcha](img/Discussion_ReCaptcha.png)

Now, if you submit the form without selecting the **I am not a robot checkbox**, you'll
be taken to the comment form page with an error message saying that the captcha
field is required. You won't be able to submit your comment until you select this box.

There are two takeaways from adding the `ReCaptcha` to our site. Firstly, notice how
easy it was for us to add a relatively complicated feature using the open source
code contributed by another programmer. Secondly, notice how—because of the
modularity that Django provides and the separation between the template and
code—all we had to do to include the `ReCaptcha` widget was add it to the form.
We didn't even need to change the view code or template. It all worked.

## Summary

This was a pretty interesting chapter. You learned a lot more about the built-in
generic view provided by Django and looked in detail into `ModelForms` and how we
could customize them. We figured out some benefits of the modular MVC pattern
that Django follows and third-party open source Django applications and how we
can include them in our projects.

You also learned a great deal about how to pass data to our forms even when they
are placed on separate pages and how to create a form that is displayed on two pages
(the comment form) while making sure that the data is synchronized between both.

All in all, the application that we ended up creating was both fun to program and a
complete product.